In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
np.random.seed(12345)

In [4]:
data = pd.read_csv('GOOG3.csv',parse_dates=True)

In [5]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-08-27,82.683998,82.750000,81.287498,81.716499,81.716499,37232000
1,2020-08-28,81.674500,82.358498,81.537498,82.220497,82.220497,29980000
2,2020-08-31,82.394501,82.398247,81.515503,81.709000,81.709000,36468000
3,2020-09-01,81.831497,83.286499,81.611000,83.035500,83.035500,36506000
4,2020-09-02,83.688751,86.658997,83.316498,86.414001,86.414001,50224000


In [6]:
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
499,2022-08-22,116.099998,116.500000,114.669998,115.070000,115.070000,19316000
500,2022-08-23,114.320000,115.930000,114.300003,114.769997,114.769997,14390700
501,2022-08-24,114.449997,115.717003,113.779999,114.699997,114.699997,16051200
502,2022-08-25,115.150002,117.779999,115.050003,117.699997,117.699997,14874700
503,2022-08-26,115.809998,116.449997,111.250000,111.300003,111.300003,31590470


In [7]:
data.shape

(504, 7)

In [8]:
data.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [9]:
data['Date'] = pd.to_datetime(data['Date'],format="%Y-%m-%d")

In [10]:
df_close = data[['Date','Adj Close']]

In [11]:
df_close.columns = ['Date','y']

In [12]:
df_close.sort_values(by='Date', inplace=True)

In [13]:
df_close.head(10)

,Date,y
0,2020-08-27,81.716499
1,2020-08-28,82.220497
2,2020-08-31,81.709000
3,2020-09-01,83.035500
4,2020-09-02,86.414001
5,2020-09-03,82.092003
6,2020-09-04,79.552002
7,2020-09-08,76.619499
8,2020-09-09,77.848000
9,2020-09-10,76.600998


In [14]:
df_close.tail(5)

,Date,y
499,2022-08-22,115.070000
500,2022-08-23,114.769997
501,2022-08-24,114.699997
502,2022-08-25,117.699997
503,2022-08-26,111.300003


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (20,6))
sns.lineplot(data=df_close, x='Date',y='y')
plt.title('Google Stock Price from Aug 2020 to Aug 2022')
plt.ylabel('Adjusted Closing Stock Price')
plt.show()

In [ ]:
# 1,2,3,4,5,6,7,8,9,10

# 1,2,3,4 -> 5
# 2,3,4,5 -> 6
# 3,4,5,6 -> 7



In [ ]:
window_size = 45
for i in range(window_size):
    df_close['y'+str(i+1)] = df_close['y'].shift(i+1)

In [ ]:
df_close.iloc[0:5,:]

In [ ]:
df_close.iloc[45:50,:]

In [ ]:
df_close.shape

In [ ]:
df_close.dropna(axis=0, inplace=True)

In [ ]:
df_close.shape

In [ ]:
df_train = df_close[df_close['Date'] <= pd.to_datetime('2022-08-14')]
df_test = df_close[df_close['Date'] > pd.to_datetime('2022-08-14')]

In [ ]:
print(f" Training shape {df_train.shape} Testing Shape {df_test.shape}")

In [ ]:
df_train.tail()

In [ ]:
df_test.head(10)

In [ ]:
X_train = df_train.drop(['Date','y'],axis=1)
y_train = df_train['y']

X_test = df_test.drop(['Date','y'],axis=1)
y_test = df_test['y']

In [ ]:
print(f" X Train Shape {X_train.shape} and X Test shape {X_test.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [ ]:
tf.random.set_seed(12345)

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(128,
               activation='relu',
               return_sequences = True,
               input_shape=(None,1)))

In [ ]:
model.add(LSTM(128,
               activation = 'relu'))

In [ ]:
model.add(Dense(units = 1,
                activation = 'linear'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='huber' ,
              metrics = 'mse')

In [ ]:
# Mape = Avg(Abs(Actual - Predicted)/Actual)
# 1 - MAPE = Accuracy

In [ ]:
cp = tf.keras.callbacks.ModelCheckpoint('GoogModel.h5',
                                        monitor='val_mse',
                                        save_weights=True,
                                        save_best_only = True,
                                        verbose=1)

In [ ]:
model.fit(X_train,
          y_train,
          validation_data = (X_test, y_test),
          epochs = 200,
          callbacks = [cp])

In [ ]:
new_model = tf.keras.models.load_model('GoogModel.h5')

In [ ]:
y_tr_pred = new_model.predict(X_train)

In [ ]:
y_ts_pred = new_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# MAPE = np.mean(np.abs(actual - predicted)/actual)

In [ ]:
print(f"Training set accuracy : {np.round(100 - mean_absolute_percentage_error(y_true = y_train, y_pred = y_tr_pred)*100,2)}%")

In [ ]:
print(f"Test set accuracy : {np.round(100 - mean_absolute_percentage_error(y_true = y_test, y_pred = y_ts_pred)*100,2)}%")

In [ ]:
df1 = df_train[['Date','y']]
df1['Pred'] = y_tr_pred

In [ ]:
df1.head()

In [ ]:
plt.figure(figsize = (20,6))
plt.plot(df1['Date'], df1['y'],color='blue', label='Actual')
plt.plot(df1['Date'], df1['Pred'],color='red', label='Predicted')
plt.title("Google Stock Price - Actual vs Predicted " )
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.legend()
plt.show()

In [ ]:
new_test_df = df_test[['Date','y']].copy()

In [ ]:
new_test_df['yhat'] = y_ts_pred

In [ ]:
new_test_df

In [ ]:
plt.figure(figsize = (12,4))
plt.plot(new_test_df['Date'], new_test_df['y'],color='blue', label='Actual')
plt.plot(new_test_df['Date'], new_test_df['yhat'],color='red', label='Predicted')
plt.title("Google Stock Price - Actual vs Predicted on Test Set" )
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.legend()
plt.show()

In [ ]:
new_test_df

In [ ]:
df_close.tail()